In [1]:
from urllib import request
from bs4 import BeautifulSoup
from urllib.request import urlopen

# Zad
Zczytaj z wikipedi artykuły o tytułach 
```python
["Integral", "Riemann_integral", "Riemann-Stieltjes_integral", "Derivative",
    "Limit_of_a_sequence", "Edvard_Munch", "Vincent_van_Gogh", "Jan_Matejko",
    "Lev_Tolstoj", "Franz_Kafka", "J._R._R._Tolkien"]
```
i zapisz w tablicy zawartości tych podstron ($id="bodyContent"$)

In [2]:
wiki = "http://en.wikipedia.org/wiki/"
titles = ["Integral", "Riemann_integral", "Riemann-Stieltjes_integral", "Derivative",
    "Limit_of_a_sequence", "Edvard_Munch", "Vincent_van_Gogh", "Jan_Matejko",
    "Lev_Tolstoj", "Franz_Kafka", "J._R._R._Tolkien"]



In [3]:
def parse(url):        
    x = urlopen(url).read()
    x = BeautifulSoup(x).find("div",id="bodyContent").find_all("p")
    return(x)

articles = [parse(url) for url in [wiki+x for x in titles]]
articles = [" ".join([str(paragraph) for paragraph in article]) for article in articles]

articles[0][:200]

C:\Users\admin\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\admin\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


'<p>In <a href="/wiki/Mathematics" title="Mathematics">mathematics</a>, an <b>integral</b> assigns numbers to functions in a way that can describe displacement, area, volume, and other concepts that ar'

In [4]:
import re
articles = [re.sub("<.+?>"," ",a) for a in articles] # ew (str(a))

In [5]:
articles[0]

' In  mathematics , an  integral  assigns numbers to functions in a way that can describe displacement, area, volume, and other concepts that arise by combining  infinitesimal  data. Integration is one of the two main operations of  calculus , with its inverse operation,  differentiation , being the other. Given a  function   f  of a  real   variable   x  and an  interval   [ a ,  b ]  of the  real line , the  definite integral \n   is defined informally as the signed  area  of the region in the  xy -plane that is bounded by the  graph  of  f , the  x -axis and the vertical lines   x  =  a   and   x  =  b  . The area above the  x -axis adds to the total and that below the  x -axis subtracts from the total.\n   The operation of integration, up to an additive constant, is the inverse of the operation of differentiation. For this reason, the term  integral  may also refer to the related notion of the  antiderivative , a function  F  whose derivative is the given function  f . In this case

In [6]:
articles[1]

' In the branch of  mathematics  known as  real analysis , the  Riemann integral , created by  Bernhard Riemann , was the first rigorous definition of the  integral  of a  function  on an  interval .  It was presented to the faculty at the  University of Göttingen  in 1854, but not published in a journal until 1868.  [1]   For many functions and practical applications, the Riemann integral can be evaluated by the  fundamental theorem of calculus  or approximated by  numerical integration .\n   The Riemann integral is unsuitable for many theoretical purposes. Some of the technical deficiencies in Riemann integration can be remedied with the  Riemann–Stieltjes integral , and most disappear with the  Lebesgue integral .\n   Let  f  be a non-negative  real -valued function on the interval  [ a ,  b ] , and let\n   be the region of the plane under the graph of the function  f  and above the interval  [ a ,  b ]  (see the figure on the top right). We are interested in measuring the area of  

# Zad.

Proszę wykonać podstawową tekenizację biorąc pod uwagę:

 * bierzemy kolejne artykuły i dzieli go na tokeny
 * bierzemy listę tokenów i usuwamy punktory
 * bierzemy listę tokenów i usuwa liczby
 * bierzemy listę tokenów i zamieniamy na małe litery

a następnie stworzyć reprezentację **Tf-Idf**.

Użyj:

```python
from sklearn.feature_extraction.text import TfidfVectorizer
```

In [7]:
from nltk.corpus import stopwords
import string
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

def remove_punctuation(words):
    remove_list = list(string.punctuation) + ["--", "\"", "``" , "...", "''", "'" ]
    return [w for w in words if not w in set(remove_list)]

def remove_nummbers(words):
    return [w for w in words if not w.isdigit()]

def to_lower(words):
    return [w.lower() for w in words]


def my_tokenizer(sent):
    tokens = word_tokenize(sent)
    tokens = remove_punctuation(tokens)
    tokens = remove_nummbers(tokens)
    tokens = to_lower(tokens)
    stemmer = PorterStemmer()
    res = [stemmer.stem(word) for word in tokens]
    return res 

tfidf_vectorizer = TfidfVectorizer(tokenizer=my_tokenizer, stop_words=stopwords.words('english') + list(string.punctuation))
tfidf_matrix = tfidf_vectorizer.fit_transform(articles)

In [8]:
print("----")
print(tfidf_matrix.todense().shape)
print(tfidf_matrix.todense())

----
(11, 6733)
[[0.         0.         0.00450557 ... 0.         0.         0.00450557]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


# Zad 
Wypisz 10 najważniejszych sów w każdym, zdaniu względem Tf-Idf

In [9]:
import numpy as np
for i in range(10):
    print(np.array(tfidf_vectorizer.get_feature_names())[
        tfidf_matrix.getrow(i).todense().argsort().A.flatten()[-10:][::-1]
    ])

['integr' 'function' 'x' 'f' 'interv' 'b' 'antideriv' 'differenti'
 'riemann' 'sum']
['integr' 'riemann' 'interv' 'ε' 'x' 'partit' 'sum' 'f' 'function' 'n']
['g' 'integr' 'riemann–stieltj' 'x' '\\displaystyl' 'function' 'f'
 'partit' 'p' 'gener']
['f' 'deriv' 'x' 'function' 'h' '′' 'differenti' 'vector' 'v' 'thi']
['n' '\\displaystyl' 'x' 'sequenc' 'x_' '\\to' 'limit' '→' 'b' 'converg']
['munch' 'hi' 'cm' '×' 'oslo' 'paint' '⁄' 'museum' 'wa' "'s"]
['gogh' 'van' 'hi' 'paint' 'theo' 'wa' "'s" 'arl' 'vincent' 'gauguin']
['matejko' 'kraków' 'hi' 'polish' 'paint' 'jan' 'wa' 'histor' 'fine'
 'finish']
['tolstoy' 'hi' "'s" 'wa' 'war' 'russian' 'anarchist' 'novel' 'peac'
 'gandhi']
['kafka' 'hi' "'s" 'brod' 'wa' 'work' 'stori' 'publish' 'der' 'jewish']


# Zad
Porównaj wyniki z najczęstszymi słowami.

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

cv_vectorizer = CountVectorizer(tokenizer=my_tokenizer, stop_words=stopwords.words('english') + list(string.punctuation))
cv_matrix = cv_vectorizer.fit_transform(articles)

In [11]:
import numpy as np
for i in range(10):
    print(np.array(cv_vectorizer.get_feature_names())[
        cv_matrix.getrow(i).todense().argsort().A.flatten()[-10:][::-1]
    ])

['integr' 'function' 'x' 'f' 'thi' 'b' 'use' 'valu' 'interv' 'sum']
['integr' 'riemann' 'x' 'sum' 'ε' 'interv' 'thi' 'partit' 'function' 'f']
['integr' 'g' 'x' 'function' 'riemann–stieltj' '\\displaystyl' 'f' 'gener'
 'continu' 'partit']
['f' 'deriv' 'function' 'x' 'thi' 'h' 'n' 'point' '′' 'differenti']
['n' '\\displaystyl' 'x' 'sequenc' 'limit' 'x_' '→' 'b' '\\to' 'f']
['munch' 'hi' 'wa' "'s" 'paint' 'cm' '×' 'museum' 'oslo' 'work']
['hi' 'van' 'gogh' 'wa' "'s" 'paint' 'work' 'theo' 'art' 'museum']
['hi' 'matejko' 'wa' 'paint' 'kraków' 'polish' "'s" 'jan' 'year' 'art']
['tolstoy' 'hi' "'s" 'wa' 'war' 'work' 'also' 'russian' 'novel'
 'anarchist']
['kafka' 'hi' "'s" 'wa' 'work' 'publish' 'stori' 'brod' 'write' 'der']


# Zad
Znajdź najbliższe zdanie z korpusu do zdań:

* "Integrals are OK"
* "I love art"

In [26]:
query = ["Integrals are OK"]

query_data = tfidf_vectorizer.transform(query)
print(query_data.todense())

res = np.sum(np.dot(tfidf_matrix, query_data.T).todense(),axis=1)
res

[[0. 0. 0. ... 0. 0. 0.]]


matrix([[0.72139115],
        [0.44981537],
        [0.47728516],
        [0.00832261],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.00177815]])

In [27]:
query = ["I love art"]

query_data = tfidf_vectorizer.transform(query)
print(query_data.todense())

res = np.sum(np.dot(tfidf_matrix, query_data.T).todense(),axis=1)
res

[[0. 0. 0. ... 0. 0. 0.]]


matrix([[0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.07483569],
        [0.05372964],
        [0.06946146],
        [0.01559913],
        [0.00772017],
        [0.01596209]])

# Zad
Znajdź najbliższe zdanie z korpusu do zdań za pomocą najczęściej występujących słów:

* "Integrals are OK"
* "I love art"

In [29]:
query = ["Integrals are OK"]

query_data = cv_vectorizer.transform(query)
print(query_data.todense())

res = np.sum(np.dot(cv_matrix, query_data.T).todense(),axis=1)
res

[[0 0 0 ... 0 0 0]]


matrix([[264],
        [110],
        [ 44],
        [  3],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  1]], dtype=int64)

In [30]:
query = ["I love art"]

query_data = cv_vectorizer.transform(query)
print(query_data.todense())

res = np.sum(np.dot(cv_matrix, query_data.T).todense(),axis=1)
res

[[0 0 0 ... 0 0 0]]


matrix([[ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [46],
        [46],
        [13],
        [ 6],
        [ 5],
        [13]], dtype=int64)